In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import string
import json
from time import time
import pickle
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, Sequential , load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM , Flatten
from keras.layers.merge import add
import os
import cv2

Using TensorFlow backend.


In [5]:
model = MobileNet(weights="imagenet",input_shape=(224,224,3))
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [6]:
model_new = Model(model.input,model.layers[-6].output)
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [7]:
def create_model(model,output_neurons):
    output_layer = Dense(units=256,activation='relu')(model.layers[-1].output)
    output_layer = Dropout(0.3)(output_layer)
    output_layer = Dense(units=output_neurons,activation='softmax')(output_layer)
    return Model(inputs = [model.input],outputs=[output_layer])

In [8]:
model = create_model(model_new,15)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [9]:
for i in range(len(model.layers)-3):
    model.layers[i].trainable = False

In [11]:
for i in range(len(model.layers)):
    print(model.layers[i].trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True


In [12]:
name_to_idx = {}
idx_to_name = {}
cnt = 0

In [14]:
for file in os.listdir('./train'):
    class_name = file.split('.')[0]
    if class_name not in name_to_idx:
        name_to_idx[class_name] = cnt
        idx_to_name[cnt] = class_name
        cnt += 1

In [15]:
print(len(idx_to_name))
print(len(name_to_idx))

4
4


In [10]:
X = []
Y = []
for file in os.listdir('./yale'):
    file_path = './yale/' + file
    name = file.split('.')[0]
    img = image.load_img(file_path,target_size=(224,224))
    img = image.img_to_array(img)
    img /= 256.0
    img = image.img_to_array(img)
    X.append(img)
    Y.append(name_to_idx[name])

Y = to_categorical(Y)
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(165, 224, 224, 3)
(165, 15)


In [1]:
model.fit(X,Y,validation_split=0.2,batch_size = 16,epochs=10)

NameError: name 'model' is not defined